In [2]:
import os
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [3]:
data_dir = os.path.join('../data/')
real_world_data_dir = os.path.join('../Dataset/')

In [4]:
os.path.join(data_dir, 'train_clip_temp.jsonl')

'../data/train_clip_temp.jsonl'

# Setup for model using clip


### 1. Create the dataset with precomputed embeddings 
- **mode 1**:
	Download the dataset from https://hatefulmemeschallenge.com/ 
	Use clip to create the jsonl with the embeddings needed for train, evaluation and test or directly download the dataset. To do so uncomment the cell below and run it.


- **mode2 (recommended)**
Computing the embeddings may take a long time to avoid that is possible to download them from [here](https://drive.google.com/drive/folders/13kap5FjN6cJ7tsckNaib1d0XC8pwfuZp?usp=sharing) and place train_clip_augmented.jsonl, eval_clip.jsonl, test_clip.jsonl, real_life_clip.jsonl directly into the data folder. This method also guarantees that the result obtained are exactly the same



In [25]:
from scripts.compute_embeddings_clip import makeClipDataset, makeClipAugmentedDataset
from core.augmentation import add_unseen
# # install clip package
# !pip install ftfy regex tqdm
# !pip install git+https://github.com/openai/CLIP.git
# # Use clip to create the jsonl with the embeddings needed for train, evaluation and test
# model_name = 'ViT-L/14'
# makeClipAugmentedDataset(os.path.join(data_dir, 'train_clip_augmented.jsonl'), os.path.join(data_dir, 'train.jsonl'), model_name)
# makeClipDataset(os.path.join(data_dir, 'eval_clip.jsonl'), os.path.join(data_dir, 'dev.jsonl'), model_name)
# makeClipDataset(os.path.join(data_dir, 'test_clip_seen.jsonl'), os.path.join(data_dir, 'test_seen.jsonl'), model_name)

# # compute the real world dataset embeddings
# makeClipDataset(os.path.join(data_dir, 'real_life_data_precomputed.jsonl'), os.path.join(real_world_data_dir, 'real_life_data.jsonl'), model_name)

# # make the augmented dataset including the unused unseen classes
add_unseen(data_dir)

file already exists, remove it to proceed: data/train_clip_temp.jsonl
file already exists, remove it to proceed: data/eval_clip_temp.jsonl
file not found: data/test_seen.jsonl


### 2. Test the model 

Run the following cell making sure that DATA_DIR in training_clip_freeze is equal to the one where you saved the files

Current implementation is based on the paper https://aclanthology.org/2022.nlp4pi-1.20.pdf implemented at https://github.com/gokulkarthik/hateclipper

The following cell run the program with the model that we indicated in the report as 'CLIP ViT-L/14 (U + A + B)' if used the pretrained embeddings from drive the results should be equal to the ones in the report.

In [2]:
!python ../training_clip_freeze.py --train_data=train_clip_augmented_all.jsonl --eval_data=eval_clip.jsonl --test_data=test_clip_seen.jsonl --paraphrase=True --augment_image=True --balancing=sampler

^C
Exception ignored in: <function _WeakValueDictionary.__init__.<locals>.KeyedRef.remove at 0x76276e7a47c0>
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 91, in remove
KeyboardInterrupt: 


The following cell run the program with the model that we indicated in the report as 'CLIP ViT-L/14 (A + B)' if used the pretrained embeddings from drive the results should be equal to the ones in the report.

In [3]:
!python ../training_clip_freeze.py --train_data=train_clip_augmented.jsonl --eval_data=eval_clip.jsonl --test_data=test_clip_seen.jsonl --paraphrase=True --augment_image=True --balancing=sampler

Using precomputed embeddings
^C
Traceback (most recent call last):
  File "/home/ceru/Documents/2o_semestre_23-24/deep_learning/hate-speach-detection/notebooks/../training_clip_freeze.py", line 248, in <module>
    main(args.model_name, args.augment_image, args.paraphrase, args.precomputed, args.balancing, args.train_data, args.eval_data, args.test_data)
  File "/home/ceru/Documents/2o_semestre_23-24/deep_learning/hate-speach-detection/notebooks/../training_clip_freeze.py", line 166, in main
    train_dataset = ClipHatefulMemeDatasetPrecomputed(os.path.join(DATA_DIR, train_data), augment_img=(augment_image=="True"), paraphrase=(paraphrase=="True"))
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ceru/Documents/2o_semestre_23-24/deep_learning/hate-speach-detection/core/dataset.py", line 158, in __init__
    self.data = [json.loads(jline) for jline in open(data_fi

The following cell run the program with the model that we indicated in the report as 'CLIP ViT-L/14 (U)' if used the pretrained embeddings from drive the results should be equal to the ones in the report.

!python ../training_clip_freeze.py --train_data=train_clip_augmented_all.jsonl --eval_data=eval_clip.jsonl --test_data=test_clip_seen.jsonl --paraphrase=False --augment_image=False --balancing=none

### Note: Run without precomputed embeddings

It is also possible to run the model without previously precomputing the embeddings. This is not particularly useful given that we used frozen clip model, and running the code this way would mean computing the embeddings 20 times (each epoch) instead of just one. In any case the following code allow to do that but the computational burden required is quite big.

In [ ]:
!python ../training_clip_freeze.py --train_data=train.jsonl --eval_data=dev_seen.jsonl --test_data=test_seen.jsonl --paraphrase=True --augment_image=True --balancing=sampler --precomputed=False